# SARSA & Q-learning

This assignment implements SARSA and Q-learning algorithms from Reinforcement Learning

In [43]:
# Python 2D Array operations - https://www.tutorialspoint.com/python/python_2darray.htm

def showMatrix(matrix):
    for row in matrix:
        for col in row:
            print(col,end = '\t')
        print()


def createMatrix(matrix_size):
    # Create an NxN matrix with all values initialized to 0
    # https://www.geeksforgeeks.org/python-using-2d-arrays-lists-the-right-way/
    squareMatrix = [[0 for i in range(matrix_size)] for j in range(matrix_size)]
    return squareMatrix

def prettifyMatrix(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            matrix[i][j] = round(matrix[i][j], 4)
    return matrix

def prettify3dMatrix(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            for k in range(len(matrix[i][j])):
                matrix[i][j][k] = round(matrix[i][j][k], 2)
    return matrix

def arbitrarilyInitializeMatrix(matrix, initial_q_range):
    # Import function to generate random integers
    # https://www.geeksforgeeks.org/python-randint-function/
    from random import randint

    # Arbitrarily initialize Q values in the format ['→', '↓', '←', '↑']
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            # https://stackoverflow.com/questions/16655089/python-random-numbers-into-a-list/16655135#16655135
            matrix[i][j] = [randint(0, initial_q_range) for count in range(4)]
    return matrix

def nextStateCoordinates(current_row, current_col, direction, grid_size):
    i = current_row
    j = current_col
    #check for out of bounds and update coordinates
    if((direction == '←') and (current_col > 0)):
        j-=1
    elif((direction == '↑') and (current_row > 0)):
        i-=1
    elif((direction == '→') and (current_col < grid_size-1)):
        j+=1
    elif(current_row < grid_size-1):
        i+=1
    return (i,j)

In [44]:
from random import uniform as probability
from random import choice as randomChoice

def findNextAction(qListOfCurrentState, epsilon):
    actionList = ['→', '↓', '←', '↑']
    # https://stackoverflow.com/questions/33359740/random-number-between-0-and-1-in-python/33359801#33359801
    if(probability(0, 1) < epsilon):
        # Explore for ε times
        # https://www.geeksforgeeks.org/random-numbers-in-python/
        return randomChoice(actionList)
    else:
        # Exploit for 1-ε times (greedy action)
        # https://stackoverflow.com/questions/2474015/getting-the-index-of-the-returned-max-or-min-item-using-max-min-on-a-list/2474030#2474030
        max_index = qListOfCurrentState.index(max(qListOfCurrentState))
        return actionList[max_index]

def sarsa(grid_size, startState, goalState, reward, discount_factor, step_size, epsilon, initial_q_range, episode_range):

    qGrid = createMatrix(grid_size)
    qGrid[startState[0]][startState[1]] = 's'
    qGrid[goalState[0]][goalState[1]] = 'x'

    print("Matrix representation of given grid world :")
    showMatrix(qGrid)
    
    qGrid = arbitrarilyInitializeMatrix(qGrid, initial_q_range)

    # Set Q values of goal state to 0
    qGrid[goalState[0]][goalState[1]] = [0,0,0,0]

    print("\nArbitrarily initialized state-action Q values in the format ['→', '↓', '←', '↑'] :")
    showMatrix(qGrid)

    print("\nIntermediate state-action Q values :")
    for episode_count in range(episode_range):
        currentState = startState
        current_action = findNextAction(qGrid[currentState[0]][currentState[1]], epsilon)

        # Simulate do-while on Python
        # https://www.javatpoint.com/python-do-while-loop
        while(True):
            nextState = nextStateCoordinates(currentState[0], currentState[1], current_action, grid_size)
            next_action = findNextAction(qGrid[nextState[0]][nextState[1]], epsilon)
            current_action_index = ['→', '↓', '←', '↑'].index(current_action)
            next_action_index = ['→', '↓', '←', '↑'].index(next_action)            
            currentQ = qGrid[currentState[0]][currentState[1]][current_action_index]
            nextQ = qGrid[nextState[0]][nextState[1]][next_action_index]
            qGrid[currentState[0]][currentState[1]][current_action_index] = currentQ + step_size * (reward + discount_factor * nextQ - currentQ)
            currentState = nextState
            current_action = next_action
            if(currentState == goalState):
                break

        print("\nEpisode", episode_count, ":")
        showMatrix(prettify3dMatrix(qGrid))

    return prettify3dMatrix(qGrid)

def qLearning(grid_size, startState, goalState, reward, discount_factor, step_size, epsilon, initial_q_range, episode_range):

    qGrid = createMatrix(grid_size)
    qGrid[startState[0]][startState[1]] = 's'
    qGrid[goalState[0]][goalState[1]] = 'x'

    print("Matrix representation of given grid world :")
    showMatrix(qGrid)
    
    qGrid = arbitrarilyInitializeMatrix(qGrid, initial_q_range)

    # Set Q values of goal state to 0
    qGrid[goalState[0]][goalState[1]] = [0,0,0,0]

    print("\nArbitrarily initialized state-action Q values in the format ['→', '↓', '←', '↑'] :")
    showMatrix(qGrid)

    print("\nIntermediate state-action Q values :")
    for episode_count in range(episode_range):
        currentState = startState

        # Simulate do-while on Python
        # https://www.javatpoint.com/python-do-while-loop
        while(True):
            current_action = findNextAction(qGrid[currentState[0]][currentState[1]], epsilon)
            current_action_index = ['→', '↓', '←', '↑'].index(current_action)
            nextState = nextStateCoordinates(currentState[0], currentState[1], current_action, grid_size)   
            currentQ = qGrid[currentState[0]][currentState[1]][current_action_index]
            maxNextQ = max(qGrid[nextState[0]][nextState[1]])
            qGrid[currentState[0]][currentState[1]][current_action_index] = currentQ + step_size * (reward + discount_factor * maxNextQ - currentQ)
            currentState = nextState
            if(currentState == goalState):
                break

        print("\nEpisode", episode_count, ":")
        showMatrix(prettify3dMatrix(qGrid))

    return prettify3dMatrix(qGrid)

In [45]:
print("SARSA\n=====\n")
stateActionMatrix = sarsa(5, (3,1), (2,3), -1, 0.625, 0.75, 0.3, 4, 8)
print("\nOptimal state-action Q values :")
showMatrix(stateActionMatrix)

print("\n\nQ-learning\n=====\n")
stateActionMatrix = qLearning(5, (3,1), (2,3), -1, 0.625, 0.75, 0.3, 4, 8)
print("\nOptimal state-action Q values :")
showMatrix(stateActionMatrix)


SARSA
=====

Matrix representation of given grid world :
0	0	0	0	0	
0	0	0	0	0	
0	0	0	x	0	
0	s	0	0	0	
0	0	0	0	0	

Arbitrarily initialized state-action Q values in the format ['→', '↓', '←', '↑'] :
[3, 1, 3, 1]	[0, 2, 1, 4]	[2, 0, 4, 4]	[4, 0, 2, 1]	[3, 0, 4, 3]	
[3, 0, 0, 4]	[4, 1, 3, 2]	[1, 0, 2, 2]	[1, 3, 1, 2]	[1, 1, 2, 0]	
[2, 1, 1, 4]	[4, 0, 2, 1]	[3, 3, 3, 4]	[0, 0, 0, 0]	[4, 4, 0, 0]	
[0, 0, 3, 2]	[0, 0, 1, 1]	[0, 1, 3, 1]	[2, 3, 2, 3]	[3, 3, 0, 2]	
[3, 4, 1, 0]	[0, 1, 3, 3]	[0, 3, 1, 0]	[4, 0, 1, 2]	[2, 3, 1, 4]	

Intermediate state-action Q values :

Episode 0 :
[3, 1, 3, 1]	[0, 2, 1, 0.72]	[2, 0, 2.12, 4]	[4, 0, 2, 1]	[3, 0, 4, 3]	
[3, 0, 0, 4]	[1.19, 0.5, 3, 2]	[1, 0, 1.62, 1.62]	[1, 3, 1, 2]	[1, 1, 2, 0]	
[2, 1, 1, 4]	[1.19, 0, 2, 1]	[0.0, 3, 3, 1.19]	[0, 0, 0, 0]	[4, 4, 0, 0]	
[0, 0, 1.41, 2]	[0, 0, 0.91, 1.38]	[0, 1, 3, 1]	[2, 3, 2, 3]	[3, 3, 0, 2]	
[1.01, 0.1, 1, 0]	[0, 1, 1.88, 0.47]	[0, 3, 1, 0]	[4, 0, 1, 2]	[2, 3, 1, 4]	

Episode 1 :
[-0.3, 0.91, 0.44, 1]	[0, -0.57, -0